In [33]:
%load_ext autoreload
%load_ext my_magics
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The my_magics extension is already loaded. To reload it, use:
  %reload_ext my_magics


In [43]:
# %%isortify
import torch
import torch.nn as nn
import utils
from data_handling.data_augmentation import VideoTransform
from data_handling.video_dataset import VideoDataset
from torch import optim
from torch.utils.data import DataLoader

from model.violence_detection_model import ViolenceDetectionModel

import torch
import torch.nn as nn
import utils
from data_handling.data_augmentation import VideoTransform
from data_handling.video_dataset import VideoDataset
from torch import optim
from torch.utils.data import DataLoader

from model.violence_detection_model import ViolenceDetectionModel



In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Initialize the model.

In [36]:
vd_model = ViolenceDetectionModel(json_file="test.json")

Test the input-output of the model.

In [37]:
vd_model.eval()

videos = torch.randn(size=(2, 20, 3, 224, 224))
lengths = torch.tensor([20, 20])

outputs = vd_model(videos, lengths)

# for each video the model outputs a label between (0, 1)
assert outputs.size() == (2,)
assert all(0 < label < 1 for label in outputs)

print("Assertion pasted")

Assertion pasted


Sanity check for the model: test its capability of memorizing a small dataset -> if the model overfits, it is capable of learning.


Check [CS231n Convolutional Neural Networks for Visual Recognition](https://cs231n.github.io/neural-networks-3/#sanitycheck) for more details.

In [38]:
print(device)

torch.cuda.empty_cache()
torch.cuda.synchronize()

vd_model.to(device)
vd_model.train()

transforms = VideoTransform(dataset="HockeyFights", json_file="test.json")
video_dataset = VideoDataset(dataset="HockeyFights", transformations=transforms, val=True, val_size=0.005)
data_loader = torch.utils.data.DataLoader(video_dataset, batch_size=5, collate_fn=utils.collate_fn_pad)

print(len(video_dataset))

criterion = nn.BCELoss()  
optimizer = optim.Adam(vd_model.parameters(), lr=0.001)

for epoch in range(40):
    
    cumulating_loss = 0
    for batch in data_loader:
        videos, labels, lengths = batch 
        videos, labels = videos.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vd_model(videos, lengths)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        cumulating_loss += loss
        
    print(f'Epoch {epoch}, Loss: {cumulating_loss / len(data_loader)}')


cuda
5
Epoch 0, Loss: 0.647121787071228
Epoch 1, Loss: 0.47743621468544006
Epoch 2, Loss: 0.5441573858261108
Epoch 3, Loss: 0.31104370951652527
Epoch 4, Loss: 0.2476349174976349
Epoch 5, Loss: 0.20075953006744385
Epoch 6, Loss: 0.1967557966709137
Epoch 7, Loss: 0.20525012910366058
Epoch 8, Loss: 0.20786228775978088
Epoch 9, Loss: 0.21972322463989258
Epoch 10, Loss: 0.13124005496501923
Epoch 11, Loss: 0.17894867062568665
Epoch 12, Loss: 0.11446704715490341
Epoch 13, Loss: 0.14833088219165802
Epoch 14, Loss: 0.1063617393374443
Epoch 15, Loss: 0.09787426143884659
Epoch 16, Loss: 0.08037625253200531
Epoch 17, Loss: 0.09254975616931915
Epoch 18, Loss: 0.07404042780399323
Epoch 19, Loss: 0.07303042709827423
Epoch 20, Loss: 0.07046137005090714
Epoch 21, Loss: 0.06582558155059814
Epoch 22, Loss: 0.08591233193874359
Epoch 23, Loss: 0.05824574455618858
Epoch 24, Loss: 0.04419877007603645
Epoch 25, Loss: 0.03685007616877556
Epoch 26, Loss: 0.0395936518907547
Epoch 27, Loss: 0.033277351409196854
E

In [41]:
vd_model.eval()

video_dataset.flag = False
print(len(video_dataset))
for batch in data_loader:
    videos, labels, lengths = batch
    videos, labels = videos.to(device), labels.to(device)
    
    outputs = vd_model(videos, lengths)
    loss = criterion(outputs, labels)

    print(f"Val: {loss.item()}")
    
    break

video_dataset.flag = True
print(len(video_dataset))
for batch in data_loader:
    videos, labels, lengths = batch
    videos, labels = videos.to(device), labels.to(device)

    outputs = vd_model(videos, lengths)
    loss = criterion(outputs, labels)

    print(f"Train: {loss.item()}")

    break

995
Val: 0.36652514338493347
5
Train: 0.184158593416214


A big difference between **train** and **val loss** indicates overfitting.